In [1]:
import pandas as pd
inf = pd.read_csv("C:\\Users\\dante\\Documents\\Downloads\\int.csv", delimiter=',', header=0)
# this is the interference data from savant
fgc = pd.read_csv("C:\\Users\\dante\\Downloads\\fgc.csv", delimiter=',', header=0)
# this is fangraphs catcher defense data

inf = inf.rename(columns={"Pitches": "Int"})

print(f'inf.head(): \n{inf.head()}\n')
print(f'fgc.head(): \n{fgc.head()}\n')

inf.head(): 
   Rk.              Player  Int   Total   Pitch %
0    1       Sánchez, Gary   12  55,596  0.000216
1    2       McCann, James    9  58,104  0.000155
2    3      Barnes, Austin    9  35,333  0.000255
3    4       Trevino, Jose    9  36,535  0.000246
4    5  Contreras, Willson    9  67,376  0.000134

fgc.head(): 
                Name   Team Pos     Inn  rSZ  rCERA  rSB  rGDP  rARM  rGFP  \
0      J.T. Realmuto  - - -   C  4748.0    1    -12   19   NaN   NaN    11   
1      Austin Hedges  - - -   C  3496.1   37     12   -2   NaN   NaN    11   
2  Christian Vazquez  - - -   C  4006.2   17      6    4   NaN   NaN    -3   
3    Yasmani Grandal  - - -   C  3788.2   27     -9   -7   NaN   NaN    -6   
4         Max Stassi  - - -   C  2710.0   28     -2   -6   NaN   NaN     1   

   ...  RngR  ErrR  UZR  UZR/150   FRM  OAA  RAA   Def  playerid  mlbamid  
0  ...   NaN   NaN  NaN      NaN  19.0  NaN  NaN  78.7     11739   592663  
1  ...   NaN   NaN  NaN      NaN  46.6  NaN  NaN  74

In [4]:
def reformat_name(name):
    last, first = name.split(", ")
    return f"{first} {last}"
inf["Name"] = inf["Player"].apply(reformat_name)

inf["Name"] = inf["Name"].str.replace("á", "a")
inf["Name"] = inf["Name"].str.replace("í", "i")
inf["Name"] = inf["Name"].str.replace("ñ", "n")
inf["Name"] = inf["Name"].str.replace("é", "e")
inf["Name"] = inf["Name"].str.replace("ó", "o")

# reformatted names to join on them

catchers = pd.merge(inf, fgc, on='Name', how='outer')
catchersdf = catchers[['Name', 'Int', 'Total', 'Pitch %', 'Inn', 'FRM']].copy()
print(catchersdf)

#Setting minimum at 100 innings caught
catchersdf = catchersdf[catchersdf['Inn'] >= 100]

num_nans = catchersdf["Int"].isna().sum()
print("Catchers with 0 Interference Calls: ", num_nans)


                   Name   Int   Total   Pitch %     Inn   FRM
0          Gary Sanchez  12.0  55,596  0.000216  3310.2  -5.6
1          James McCann   9.0  58,104  0.000155  3504.1 -12.4
2         Austin Barnes   9.0  35,333  0.000255  2269.0  22.1
3          Jose Trevino   9.0  36,535  0.000246  2201.1  31.1
4     Willson Contreras   9.0  67,376  0.000134  4008.1 -34.3
..                  ...   ...     ...       ...     ...   ...
206        Nick Hundley   NaN     NaN       NaN   837.1 -12.9
207      Salvador Perez   NaN     NaN       NaN  2939.1 -45.2
208        Chance Sisco   NaN     NaN       NaN  1142.2 -20.1
209  Isiah Kiner-Falefa   NaN     NaN       NaN   586.0 -17.9
210         Riley Adams   NaN     NaN       NaN   649.0 -14.5

[211 rows x 6 columns]
Catchers with 0 Interference Calls:  62


In [7]:
catchersdf['Int'] = catchersdf['Int'].fillna(0).astype(int)
sort_catchersdf = catchersdf[['Name', 'Int','Inn', 'FRM']].copy()
sort_catchersdf = sort_catchersdf.sort_values(by='FRM', ascending=False)

sort_catchersdf

,Name,Int,Inn,FRM
34,Austin Hedges,3,3496.1,46.6
52,Max Stassi,2,2710.0,38.1
7,Yasmani Grandal,6,3788.2,35.1
97,Christian Vazquez,0,4006.2,31.8
3,Jose Trevino,9,2201.1,31.1
...,...,...,...,...
39,Kurt Suzuki,3,2536.2,-27.6
12,Elias Diaz,6,3237.1,-32.1
10,Robinson Chirinos,6,2769.1,-33.2
4,Willson Contreras,9,4008.1,-34.3


In [13]:
#let's build a regression

import statsmodels.api as sm

X = catchersdf['FRM']
y = catchersdf['Int']
X = sm.add_constant(X) 

model = sm.OLS(y, X).fit()  
print(model.summary())  

                            OLS Regression Results                            
Dep. Variable:                    Int   R-squared:                       0.009
Model:                            OLS   Adj. R-squared:                  0.002
Method:                 Least Squares   F-statistic:                     1.348
Date:                Sat, 13 May 2023   Prob (F-statistic):              0.247
Time:                        14:01:53   Log-Likelihood:                -347.96
No. Observations:                 157   AIC:                             699.9
Df Residuals:                     155   BIC:                             706.0
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          1.7389      0.178      9.753      0.0

In [15]:
X = catchersdf[['FRM', 'Inn']]
y = catchersdf['Int']
X = sm.add_constant(X) 

model = sm.OLS(y, X).fit()  
print(model.summary())  

                            OLS Regression Results                            
Dep. Variable:                    Int   R-squared:                       0.278
Model:                            OLS   Adj. R-squared:                  0.268
Method:                 Least Squares   F-statistic:                     29.58
Date:                Sat, 13 May 2023   Prob (F-statistic):           1.35e-11
Time:                        14:02:24   Log-Likelihood:                -323.12
No. Observations:                 157   AIC:                             652.2
Df Residuals:                     154   BIC:                             661.4
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.4075      0.233      1.750      0.0